In [2]:
# Tensorflow
import tensorflow as tf
# Image data and layers pacakges
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Reshape, LeakyReLU, Dropout, UpSampling2D, MaxPool2D, Rescaling
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom
# Losses and Optimizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

# Base model class to subclass for training step
from tensorflow.keras.models import Model

# Load images from directory
from tensorflow.keras.utils import image_dataset_from_directory
# For callbacks and checkpoints
from tensorflow.keras.callbacks import Callback
from tensorflow.train import Checkpoint
# Extra
import numpy as np
import os

In [3]:
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
os.chdir("C:\\GANs\\zelda_gan\\code")
resize_path = "..\\resized_images"

In [5]:
batch_size = 32
img_height = 40
img_width = 30

In [6]:
def load_images(in_dir, img_height, img_width, batch_size, save_dir):
    norm = lambda x: x.astype("float32")/255
    dg = ImageDataGenerator(preprocessing_function = img_norm, zoom_range = 0.25, 
                                  horizontal_flip = True, rotation_range = 0.05)
    x_train = dg.flow_from_directory(in_dir,
                                            target_size = (img_height, img_width),
                                            batch_size = batch_size,
                                            shuffle = True,
                                            save_to_dir = save_dir,
                                            classes = None,
                                            class_mode = None,
                                            subset = "training")
    return x_train

In [7]:

img_norm = tf.keras.layers.Rescaling(1./255)
train_ds = image_dataset_from_directory(resize_path, 
                                       labels = None, 
                                       color_mode = 'grayscale', 
                                       image_size = (img_height , img_width),
                                       batch_size = batch_size, 
                                       shuffle = True)
train_ds = train_ds.map(img_norm)
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 2340 files belonging to 1 classes.


In [8]:
# Generative NN
def build_gen_nn():
    # Initialize Model
    model = Sequential()

    # Takes in random values and reshape it to 7x7x64
    model.add(Dense(20*15*128, input_dim = 64))
    model.add(LeakyReLU(.2))
    model.add(Reshape((20, 15, 128)))

    # Effectively doubles the size of previous layer
    model.add(UpSampling2D())
    model.add(Conv2D(128, 5, padding = "same"))
    model.add(LeakyReLU(.2))

    # Second Upsampling block
    model.add(UpSampling2D())
    model.add(Conv2D(256, 5, padding = "same"))
    model.add(LeakyReLU(.2))
    
        
    # Down Sample
    model.add(MaxPool2D())
    model.add(Conv2D(256, 5, padding = "same"))
    model.add(LeakyReLU(.2))
    
    # Normal Convolutional Block
    model.add(Conv2D(256, 4, padding = "same"))
    model.add(LeakyReLU(.2))
    
    # Normal Convolutional Block
    model.add(Conv2D(512, 4, padding = "same"))
    model.add(LeakyReLU(.2))
    
    # Second Convolutional Block
    model.add(Conv2D(512, 4, padding = "same"))
    model.add(LeakyReLU(.2))


    # Conv layer to get to one channel
    model.add(Conv2D(1, 4, padding = 'same', activation = "sigmoid"))
    
    return model

In [9]:
def build_disc_nn():
    model = Sequential()

    model.add(Conv2D(32, 5, input_shape = (40,30,1)))
    model.add(LeakyReLU(.2))
    model.add(Dropout(.2))

    model.add(Conv2D(64, 5))
    model.add(LeakyReLU(.2))
    model.add(Dropout(.2))

    model.add(Conv2D(128, 5))
    model.add(LeakyReLU(.2))
    model.add(Dropout(.2))

    model.add(Conv2D(256, 5))
    model.add(LeakyReLU(.2))
    model.add(Dropout(.2))
    
    model.add(Conv2D(512, 5))
    model.add(LeakyReLU(.2))
    model.add(Dropout(.2))

    model.add(Conv2D(512, 5))
    model.add(LeakyReLU(.2))
    model.add(Dropout(.2))

    # Flatten then pass to a dense layer
    model.add(Flatten())
    model.add(Dropout(.4))
    model.add(Dense(1, activation = "sigmoid"))

    return model

In [10]:
class ZelGAN(Model):
    def __init__(self, generator, discriminator, *args, **kwargs):
        # pass args and kwards to base class
        super().__init__(*args, **kwargs)
        
        # Create attributes for two models
        self.generator = generator
        self.discriminator = discriminator
        self.transformation = Sequential([
  RandomFlip("horizontal_and_vertical"),
  RandomRotation(0.2),
  RandomZoom(height_factor=(-0.1, 0.1), width_factor = (-0.1, 0.1))
])

        pass
    
    def compile(self, gen_opt, disc_opt, gen_loss, disc_loss, *args, **kwargs):
        # Compile with base class
        super().compile(*args, **kwargs)

        # Create attributes for losses and optimizers
        self.gen_opt = gen_opt
        self.disc_opt = disc_opt
        self.gen_loss = gen_loss
        self.disc_loss = disc_loss
    
    def train_step(self, batch):
        # Get the data
        real_images = self.transformation(batch)
        gen_images = self.generator(tf.random.normal((32, 64, 1), 0, 2), training = False)
        # Train the discriminator
        with tf.GradientTape() as disc_tape:
            # Pass the real and generated images to the discriminator
            yhat_real = self.discriminator(real_images, training = True)
            yhat_gen = self.discriminator(gen_images, training = True)
            yhat_all = tf.concat([yhat_real, yhat_gen], axis = 0)

            # Creat labels for real and generated images
            y_all = tf.concat([tf.zeros_like(yhat_real), tf.ones_like(yhat_gen)], axis = 0)

            # Add some noise to the TRUE outputs
            noise_real = .1 * tf.random.normal(tf.shape(yhat_real), 0, 2)
            noise_gen = -.1 * tf.random.normal(tf.shape(yhat_gen), 0, 2)
            y_all += tf.concat([noise_real, noise_gen], axis = 0)

            # Calculate loss
            total_disc_loss = self.disc_loss(y_all, yhat_all)

        # Apply backprop
        disc_grad = disc_tape.gradient(total_disc_loss, self.discriminator.trainable_variables)
        self.disc_opt.apply_gradients(zip(disc_grad, self.discriminator.trainable_variables))

        with tf.GradientTape() as gen_tape:
            # Generate some new images
            gen_images = self.generator(tf.random.normal((64, 64, 1), 0, 2), training = True)
            
            # Create the predicted labels
            predicted_labels = self.discriminator(gen_images, training = False)
            
            # Calculate Loss
            total_gen_loss = self.gen_loss(tf.zeros_like(predicted_labels), predicted_labels)
            
        # Apply Backprop
        gen_grad = gen_tape.gradient(total_gen_loss, self.generator.trainable_variables)
        self.gen_opt.apply_gradients(zip(gen_grad, self.generator.trainable_variables))
        
        

        return {"disc_loss" : total_disc_loss, "gen_loss" : total_gen_loss}

In [11]:
# Initialize Networks
generator = build_gen_nn()
discriminator = build_disc_nn()

# Set up Losses and Optimizers
gen_opt = Adam(learning_rate = .000005, beta_1=0.8, clipvalue=1.0)
disc_opt = Adam(learning_rate = .0000005, beta_1=0.8, clipvalue=1.0)
gen_loss = BinaryCrossentropy()
disc_loss = BinaryCrossentropy()


In [12]:
Zelda = ZelGAN(generator, discriminator)
Zelda.compile(gen_opt, disc_opt, gen_loss, disc_loss)

In [13]:
class ModelMonitor(Callback):
    def __init__(self, num_img = 1, latent_dim = 64):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs = None):
        random_latent_vectors = tf.random.normal((self.num_img, self.latent_dim, 1), 0, 2)
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        for i in range(self.num_img):
            img = array_to_img(generated_images[i])
            img.save(os.path.join("..\\progress_images", f"generated_img_{epoch}_{i}.png"))
        


In [14]:
Zelda.generator.load_weights('..//saves//generator_norm.h5')
Zelda.discriminator.load_weights('..//saves//discriminator_norm.h5')

In [ ]:
with tf.device('/GPU:1'):
    hist = Zelda.fit(train_ds, epochs = 512, callbacks = [ModelMonitor()])

Epoch 1/512
74/74 [==============================] - 93s 1s/step - disc_loss: 0.6873 - gen_loss: 0.6616
Epoch 2/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6890 - gen_loss: 0.6608
Epoch 3/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6887 - gen_loss: 0.6599
Epoch 4/512
74/74 [==============================] - 84s 1s/step - disc_loss: 0.6909 - gen_loss: 0.6594
Epoch 5/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6896 - gen_loss: 0.6587
Epoch 6/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6898 - gen_loss: 0.6580
Epoch 7/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6898 - gen_loss: 0.6572
Epoch 8/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6887 - gen_loss: 0.6566
Epoch 9/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6907 - gen_loss: 0.6561
Epoch 10/512
74/74 [==============================] - 82s 1s/ste

74/74 [==============================] - 82s 1s/step - disc_loss: 0.6957 - gen_loss: 0.6553
Epoch 80/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6953 - gen_loss: 0.6562
Epoch 81/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6954 - gen_loss: 0.6567
Epoch 82/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6946 - gen_loss: 0.6571
Epoch 83/512
74/74 [==============================] - 83s 1s/step - disc_loss: 0.6948 - gen_loss: 0.6578
Epoch 84/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6951 - gen_loss: 0.6584
Epoch 85/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6951 - gen_loss: 0.6594
Epoch 86/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6952 - gen_loss: 0.6596
Epoch 87/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6947 - gen_loss: 0.6603
Epoch 88/512
74/74 [==============================] - 82s 1s/step - 

74/74 [==============================] - 82s 1s/step - disc_loss: 0.6860 - gen_loss: 0.6735
Epoch 157/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6845 - gen_loss: 0.6734
Epoch 158/512
74/74 [==============================] - 83s 1s/step - disc_loss: 0.6823 - gen_loss: 0.6738
Epoch 159/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6794 - gen_loss: 0.6740
Epoch 160/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6771 - gen_loss: 0.6730
Epoch 161/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6729 - gen_loss: 0.6715
Epoch 162/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6718 - gen_loss: 0.6701
Epoch 163/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6681 - gen_loss: 0.6681
Epoch 164/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6648 - gen_loss: 0.6656
Epoch 165/512
74/74 [==============================] - 82s 1

74/74 [==============================] - 83s 1s/step - disc_loss: 0.6994 - gen_loss: 0.6313
Epoch 234/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6996 - gen_loss: 0.6366
Epoch 235/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6979 - gen_loss: 0.6381
Epoch 236/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6984 - gen_loss: 0.6394
Epoch 237/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6986 - gen_loss: 0.6379
Epoch 238/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6969 - gen_loss: 0.6416
Epoch 239/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6993 - gen_loss: 0.6435
Epoch 240/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6981 - gen_loss: 0.6452
Epoch 241/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6982 - gen_loss: 0.6460
Epoch 242/512
74/74 [==============================] - 82s 1

74/74 [==============================] - 82s 1s/step - disc_loss: 0.7030 - gen_loss: 0.5566
Epoch 311/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.7028 - gen_loss: 0.5614
Epoch 312/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.7031 - gen_loss: 0.5691
Epoch 313/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.7006 - gen_loss: 0.5756
Epoch 314/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.7023 - gen_loss: 0.5829
Epoch 315/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6999 - gen_loss: 0.5923
Epoch 316/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6990 - gen_loss: 0.5985
Epoch 317/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6980 - gen_loss: 0.6040
Epoch 318/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.6949 - gen_loss: 0.6094
Epoch 319/512
74/74 [==============================] - 82s 1

74/74 [==============================] - 82s 1s/step - disc_loss: 0.9911 - gen_loss: 0.8216
Epoch 388/512
74/74 [==============================] - 82s 1s/step - disc_loss: 1.2110 - gen_loss: 1.1673
Epoch 389/512
74/74 [==============================] - 82s 1s/step - disc_loss: 1.0749 - gen_loss: 0.8109
Epoch 390/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.9919 - gen_loss: 0.7902
Epoch 391/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.9260 - gen_loss: 0.8127
Epoch 392/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.8908 - gen_loss: 0.8473
Epoch 393/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.8514 - gen_loss: 0.8486
Epoch 394/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.8149 - gen_loss: 0.8606
Epoch 395/512
74/74 [==============================] - 82s 1s/step - disc_loss: 0.7929 - gen_loss: 0.8789
Epoch 396/512
74/74 [==============================] - 82s 1

In [ ]:
Zelda.generator.save('..//saves//generator_norm.h5')
Zelda.discriminator.save('..//saves//discriminator_norm.h5')

In [ ]:
#random_latent_vectors = tf.random.normal((50, 64, 1))
#generated_images = Zelda.generator(random_latent_vectors)
#generated_images *= 255
#generated_images.numpy()
#for i in range(50):
#    img = array_to_img(generated_images[i])
#    img.save(os.path.join("..\\progress_images", f"generated_img_{i}.png"))